### 1 - Import test data

In [1]:
import pandas as pd
df=pd.read_csv("BTCTUSD_1h.csv")
df
df = df.iloc[:, :6]

df.columns = ['Gmt time',  'Open', 'High', 'Low', 'Close','Volume']
df
df["Gmt time"]=df["Gmt time"].str.replace(".000","")
df['Gmt time']=pd.to_datetime(df['Gmt time'])
df.set_index("Gmt time", inplace=True)
df=df[df.High!=df.Low]
len(df)

43446

In [2]:
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

class BreakoutStrategy(Strategy):
    atrPeriod = 24
    atrMultiplier = 1
    longTrade = True
    shortTrade = True
    InpRisk = 1.0
    startHour = 1
    endHour = 23
    
    def init(self):
        super().init()
        self.atr = self.I(ATR, self.data.High, self.data.Low, self.data.Close, self.atrPeriod)
        
    def next(self):
        if not self.startHour <= self.data.index.hour <= self.endHour:
            return
        
        atrValue = self.atr[-1] * self.atrMultiplier
        upperLevel = self.data.High[-2] + atrValue
        lowerLevel = self.data.Low[-2] - atrValue
        
        price = (self.data.Close[-1] + self.data.Open[-1]) / 2
        stopDistance = abs(upperLevel - lowerLevel)
        
        if self.longTrade and price > upperLevel:
            self.buy(sl=lowerLevel)
        elif self.shortTrade and price < lowerLevel:
            self.sell(sl=upperLevel)

def ATR(high, low, close, n):
    return pd.Series((high - low).rolling(window=n, min_periods=1).mean())

bt = Backtest(df, BreakoutStrategy, cash=10000, commission=.002)
stats = bt.run()
print(stats)


C:\Users\vnarz\AppData\Local\Temp\ipykernel_25752\1934131328.py:37: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, BreakoutStrategy, cash=10000, commission=.002)


RuntimeError: Indicator "ATR(H,L,C,24)" errored with exception: '_Array' object has no attribute 'rolling'